In [ ]:
# numpy for numerics, sympy for algebra, plotly and matplotlib for plots
import numpy as np
import cupy as cp
from sympy import *
from sympy.abc import x, t
from IPython.display import display, Math
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import os

# some image processing that locates atoms
from skimage.feature import peak_local_max
from scipy.ndimage import center_of_mass

# writing data to files
import h5py

plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 10})

# Gaussian Derivative using FFT

## 1D

Calculate the derivative of a 1D FFT numerically and analytically and plot to compare.

In [ ]:
class Gaussian1D_Analytic:
  def __init__(self):
    self.x, self.x0, self.sigma = symbols('x x0 sigma')
    self.n = exp(-(self.x-self.x0)**2/(2*self.sigma**2))
    self.dndx = diff(self.n, self.x).simplify()
    self.d2ndx2 = diff(self.n, self.x, 2).simplify()

gaussian1D_Analytic = Gaussian1D_Analytic()
Math(r'n = {}, '.format(latex(gaussian1D_Analytic.n))+r'dn/dx = {}, '.format(latex(gaussian1D_Analytic.dndx))+r'd^2n/dx^2 = {}'.format(latex(gaussian1D_Analytic.d2ndx2)))

For $n = e^{-(x-x_0)^2/(2 \sigma^2)}$, $dn/dx = -(x - x_0)/\sigma^2 \cdot n$.

### Parameters

In [ ]:
class Gaussian1D_Parms:
  def __init__(self):
    self.x0 = 1
    self.xmin = -10
    self.xmax = 10
    self.sigma = 2

### Analytic Derivative

In [ ]:
class Gaussian1D_AnalyticNumeric:
  def __init__(self):
    self.analytic = Gaussian1D_Analytic()
    self.parms = Gaussian1D_Parms()
    self.pts = 1000
    self.x = np.linspace(self.parms.xmin, self.parms.xmax, self.pts)
    self.dxdt_num = lambdify(self.analytic.x, self.analytic.dndx.subs([(self.analytic.x0, self.parms.x0), (self.analytic.sigma, self.parms.sigma)]))

  def Get_dxdt(self):
    return self.dxdt_num(self.x)

gaussian1D_AnalyticNumeric = Gaussian1D_AnalyticNumeric()
plt.plot(gaussian1D_AnalyticNumeric.x, gaussian1D_AnalyticNumeric.Get_dxdt())
plt.xlabel('x')
plt.ylabel('$dn/dx, n=e^{-(x-x_0)^2/(2 \sigma^2)}$')
plt.title('Analytic Derivative')
plt.show()

### Numeric Derivative via FFT

In [ ]:
class Gaussian1D_Numeric:
  def __init__(self):
    self.parms = Gaussian1D_Parms()
    self.LX  = 100
    self.dx = (self.parms.xmax-self.parms.xmin)/self.LX
    self.x = np.linspace(self.parms.xmin, self.parms.xmax, self.LX)
    self.phi = np.exp(-(self.x-self.parms.x0)**2/(2*self.parms.sigma**2))

  def GetNumericDerivative(self):
    kappa = 2*np.pi*np.fft.fftshift(np.fft.fftfreq(self.phi.shape[0], d=self.dx))
    phi_hat = np.fft.fftshift(np.fft.fft(self.phi))
    dphi_hat = I*kappa*phi_hat
    dphi = np.fft.ifft(np.fft.ifftshift(dphi_hat))
    return dphi

### Compare Analytic and Numeric by Plotting

In [ ]:
gaussian1D_Numeric = Gaussian1D_Numeric()
dxdt_num = gaussian1D_Numeric.GetNumericDerivative()
plt.scatter(gaussian1D_Numeric.x, gaussian1D_Numeric.GetNumericDerivative(), label='Numeric')
plt.plot(gaussian1D_AnalyticNumeric.x, gaussian1D_AnalyticNumeric.Get_dxdt(), label='Analytic')
plt.xlabel('x')
plt.ylabel('$dn/dx, n=e^{-(x-x_0)^2/(2 \sigma^2)}$')
plt.title('Analytic Derivative vs. Numeric Derivative')
plt.legend()
plt.show()

## 2D

Calculate the derivative of a 1D FFT numerically and analytically and plot to compare.

In [ ]:
class Gaussian2D_Analytic:
  def __init__(self):
    self.x, self.y, self.x0, self.y0, self.sigma = symbols('x y x0 y0 sigma')
    self.n = exp(-((self.x-self.x0)**2 + (self.y-self.y0)**2)/(2*self.sigma**2))
    self.d2ndx2 = (diff(self.n, self.x, 2) + diff(self.n, self.y, 2)).simplify()
    self.d4ndx4 = (diff(self.d2ndx2, self.x, 2) + diff(self.d2ndx2, self.y, 2)).simplify()

gaussian2D_Analytic = Gaussian2D_Analytic()
Math(r'n = {}, '.format(latex(gaussian2D_Analytic.n))+r'd^2n/dx^2 = {}, '.format(latex(gaussian2D_Analytic.d2ndx2))+r'd^4n/dx^4 = {}'.format(latex(gaussian2D_Analytic.d4ndx4)))

### Parameters

In [ ]:
class Gaussian2D_Parms:
  def __init__(self):
    self.x0 = 1
    self.y0 = 0.5
    self.xmin = -10
    self.xmax = 10
    self.ymin = -10
    self.ymax = 10
    self.sigma = 2
    self.pts = 100

### Analytic Derivative

In [ ]:
class Gaussian2D_AnalyticNumeric:
  def __init__(self):
    self.analytic = Gaussian2D_Analytic()
    self.parms = Gaussian2D_Parms()
    self.Init()

  def Init(self):
    self.pts = self.parms.pts
    self.x = cp.linspace(self.parms.xmin, self.parms.xmax, self.pts)
    self.y = cp.linspace(self.parms.ymin, self.parms.ymax, self.pts)
    self.X, self.Y = cp.meshgrid(self.x, self.y)
    self.d2ndx2_num = lambdify((self.analytic.x, self.analytic.y), self.analytic.d2ndx2.subs([(self.analytic.x0, self.parms.x0), (self.analytic.y0, self.parms.y0), (self.analytic.sigma, self.parms.sigma)]))
    self.d4ndx4_num = lambdify((self.analytic.x, self.analytic.y), self.analytic.d4ndx4.subs([(self.analytic.x0, self.parms.x0), (self.analytic.y0, self.parms.y0), (self.analytic.sigma, self.parms.sigma)]))

  def Get_d2dx2(self):
    return self.d2ndx2_num(self.X, self.Y)#.reshape(self.pts, self.pts)

  def Get_d4dx4(self):
    return self.d4ndx4_num(self.X, self.Y)#.reshape(self.pts, self.pts)

gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_AnalyticNumeric.X.get()
Y = gaussian2D_AnalyticNumeric.Y.get()
Z = gaussian2D_AnalyticNumeric.Get_d2dx2().get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Second Derivative}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()


In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_AnalyticNumeric.X.get()
Y = gaussian2D_AnalyticNumeric.Y.get()
Z = gaussian2D_AnalyticNumeric.Get_d4dx4().get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^4 \phi \text{: 2D Guassian Fourth Derivative}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()


### Numeric Derivative via FFT

In [ ]:
class Gaussian2D_Numeric:
  def __init__(self):
    self.parms = Gaussian2D_Parms()
    self.Init()

  def Init(self):
    # Define grid
    self.nx = self.parms.pts
    self.ny = self.parms.pts
    self.dx = (self.parms.xmax-self.parms.xmin)/self.nx
    self.dy = (self.parms.ymax-self.parms.ymin)/self.ny
    x = cp.linspace(self.parms.xmin, self.parms.xmax, self.nx)
    y = cp.linspace(self.parms.ymin, self.parms.ymax, self.ny)
    self.x, self.y = cp.meshgrid(x, y)

    # Create the wave numbers
    kx = cp.fft.fftfreq(self.nx, d=self.dx) * 2 * cp.pi
    ky = cp.fft.fftfreq(self.ny, d=self.dy) * 2 * cp.pi
    self.kx, self.ky = cp.meshgrid(kx, ky)
    self.k2 = self.kx**2 + self.ky**2
    self.k4 = self.k2**2#self.kx**4 + self.ky**4 + 2*(self.kx**2 + self.ky**2)

    self.phi = cp.exp(-((self.x-self.parms.x0)**2 + (self.y-self.parms.y0)**2)/(2*self.parms.sigma**2))

    self.Analytic = Gaussian2D_AnalyticNumeric()
    self.Analytic.parms.pts = self.nx
    self.Analytic.Init()

  def Get_d2dx2(self):
    return self.Analytic.Get_d2dx2()

  def Get_d4dx4(self):
    return self.Analytic.Get_d4dx4()

  def Get_nabla2_Fourier(self, phi):
    phi_hat = cp.fft.fft2(phi)
    d2phi_hat = -self.k2*phi_hat
    d2phi = cp.fft.ifft2(d2phi_hat)
    return d2phi.real#.get()

  def Get_nabla4_Fourier(self, phi):
    phi_hat = cp.fft.fft2(phi)
    d4phi_hat = +self.k4*phi_hat
    d4phi = cp.fft.ifft2(d4phi_hat)
    return d4phi.real#.get()

  def Get_nabla2_FiniteDiff(self, phi):
    return ((cp.roll(phi, 1, axis=0) + cp.roll(phi, -1, axis=0) - 2*phi)/self.dx**2 + (cp.roll(phi, 1, axis=1) + cp.roll(phi, -1, axis=1) - 2*phi)/self.dy**2)#.get()

  def Get_nabla4_FiniteDiff(self, phi):
    return (
        (cp.roll(phi, 2, axis=0) - 4 * cp.roll(phi, 1, axis=0) + 6 * phi - 4 * cp.roll(phi, -1, axis=0) + cp.roll(phi, -2, axis=0))/self.dx**4 +
        (cp.roll(phi, 2, axis=1) - 4 * cp.roll(phi, 1, axis=1) + 6 * phi - 4 * cp.roll(phi, -1, axis=1) + cp.roll(phi, -2, axis=1))/self.dy**4
    )#.get()

gaussian2D_numeric = Gaussian2D_Numeric()

print(gaussian2D_numeric.dx)

In [ ]:
%timeit gaussian2D_numeric.Get_nabla2_Fourier(gaussian2D_numeric.phi)
%timeit gaussian2D_numeric.Get_nabla4_Fourier(gaussian2D_numeric.phi)

%timeit gaussian2D_numeric.Get_nabla4_FiniteDiff(gaussian2D_numeric.phi)
%timeit gaussian2D_numeric.Get_nabla4_FiniteDiff(gaussian2D_numeric.phi)

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla2_Fourier(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot_r')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Second Derivative - Fourier}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla2_FiniteDiff(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot_r')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Second Derivative - Finite Differences}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla4_Fourier(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Fourth Derivative - Fourier}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

In [ ]:
gaussian2D_AnalyticNumeric = Gaussian2D_AnalyticNumeric()
gaussian2D_AnalyticNumeric.parms.pts = 300
gaussian2D_AnalyticNumeric.Init()

X = gaussian2D_numeric.x.get()
Y = gaussian2D_numeric.y.get()
Z = gaussian2D_numeric.Get_nabla4_FiniteDiff(gaussian2D_numeric.phi).get()

fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y, colorscale='hot')])

# Update layout for orthographic projection
fig.update_layout(
    scene = dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectratio=dict(x=1, y=1, z=1),  # Adjust aspect ratio if needed
        camera_projection_type="orthographic"
    ),
    title=r'$\nabla^2 \phi \text{: 2D Guassian Fourth Derivative - Finite Differences}$',
    autosize=False,
    width=800,
    height=800
)

fig.show()

# Vacancy PFC

Energy function $f$:

$$
f(\rho) = \frac{\rho}{2} \left[ r + (1 + \nabla^2)^2 \right] \rho + \frac{\rho^4}{4}
$$

Then energy functional $F$:

$$
F(\rho) = \int d \vec{x} \left( \frac{\rho}{2} \left[ r + (1 + \nabla^2)^2 \right] \rho + \frac{\rho^4}{4} \right)
$$

And the Caan-Hilliard equation:

$$
\frac{\partial \rho}{\partial t} = \nabla^2 \frac{\delta F}{\delta \rho} = \nabla^2 \left[ (r+1)\rho + \rho^3 + 2 \nabla^2 \rho + \nabla^4 \rho \right]
$$

In Fourier space gives:

$$
\frac{\partial \hat{\rho}}{\partial t} = -\left[ k^6 - 2k^4 + (r+1)k^2 \right] \hat{\rho} - k^2 \hat{(\rho^3)}
$$

## Model

In [ ]:
class PFC2D_Vacancy_Parms:
  def __init__(self):
    self.r = None                       # corresponds to temperature
    self.a = None
    self.b = None
    self.t = None
    self.v = None
    self.Hln = None
    self.Hng = None
    self.N = None                       # number of pixels in the square region
    self.PPU = None                     # pixels per unit
    self.rho0 = None                    # the average density of the initial field
    self.eta = None                     # the strength of the gaussian noise added to each time step
    self.dt = None                      # time step
    self.seed = 0                       # set for reproducibility
    self.noise_dynamics = False         # set to True to add noise to the dynamics
    self.noise_high_freq_cutoff = 0.5   # high frequency threshold for noise
    self.time_smooth_noise_dynamics = False
                                        # set to True to make noise vary consistantly with time
    self.noise_time_smooth_steps = 30   # number of time steps to smooth the noise
    self.noise_time_smooth = 0.1        # time scale for smoothing the noise

    self.Vb = None                      # boundary potential
    self.phi_b = None                   # boundary density

class PFC2D_Vacancy:
  def __init__(self):
    self.t = None      # elapsed time
    self.parms = PFC2D_Vacancy_Parms()
    self.minLog = -30

  def InitParms(self):
    # Set seed for reproducibility
    cp.random.seed(cp.uint64(self.parms.seed))
    
    # Initialize geometry
    self.SetGeometry(self.parms.N, self.parms.N, self.parms.PPU, 1)

    # SetDT
    self.SetDT(self.parms.dt)

  def SetGeometry(self, NX, NY, PPUx, scalefactor):
    self.nx = NX
    self.ny = NY
    self.mx = int(np.round(NX/PPUx))
    self.my = int(np.round(NY/(np.sqrt(3)*PPUx)))*2
    self.dx = (4*np.pi*self.mx/np.sqrt(3))/NX
    self.dy = (2*np.pi*self.my)/NY
    self.dy *= scalefactor
    self.dx *= scalefactor
    x = cp.arange(NX) * self.dx
    y = cp.arange(NY) * self.dy
    self.x, self.y = cp.meshgrid(x,y)

    # using self.x, self.y create an array of ordered pairs
    self.r = cp.dstack((self.x, self.y))

    # Create the wave numbers
    kx = cp.fft.fftfreq(self.nx, d=self.dx) * 2 * cp.pi
    ky = cp.fft.fftfreq(self.ny, d=self.dy) * 2 * cp.pi
    self.kx, self.ky = cp.meshgrid(kx, ky)
    self.k2 = self.kx**2 + self.ky**2
    self.k4 = self.k2**2
    self.k6 = self.k2**3
    self.lincoeff = (-self.k2 * (self.parms.r + self.parms.b) + 2 * self.parms.b * self.k4 - self.parms.b * self.k6)
    self.linenergycoeff = self.parms.b * self.k4 - self.parms.b * self.k6 / 2

    # Create noise mask.
    self.noiseMask = cp.ones((self.ny, self.nx))

    # SetDT
    self.SetDT(self.parms.dt)

  def SetDT(self, dt):
    self.parms.dt = dt
    self.expcoeff = cp.exp(self.lincoeff*self.parms.dt)

    self.expcoeff_nonlin = cp.ones_like(self.x) * self.parms.dt
    self.expcoeff_nonlin[self.lincoeff != 0] = (self.expcoeff[self.lincoeff != 0] - 1)/self.lincoeff[self.lincoeff != 0]

    self.expcoeff_nonlin2 = cp.zeros_like(self.x)
    self.expcoeff_nonlin2[self.lincoeff != 0] = (self.expcoeff[self.lincoeff != 0] - (1 + self.lincoeff[self.lincoeff != 0]*self.parms.dt))/cp.power(self.lincoeff[self.lincoeff != 0],2)

  def GetEtaNoise(self):
    # noise = self.noiseMask*cp.random.normal(loc=0, scale=self.parms.eta, size=(self.ny, self.nx))
    # noise_fft = 1j*self.kx*cp.fft.fft2(noise)
    # self.noise_fft = noise_fft.copy()
    # noise = self.noiseMask*cp.random.normal(loc=0, scale=self.parms.eta, size=(self.ny, self.nx))
    # noise_fft = 1j*self.ky*cp.fft.fft2(noise)

    # # high frequency cutoff
    # self.noise_fft += noise_fft
    # self.noise_fft[0,0] = 0
    # self.noise_fft[self.k2 > self.parms.noise_high_freq_cutoff**2] = 0
    
    # # inverse fourier to get normalized noise
    # # noise_fft[0,0] = 0
    # self.noise = cp.fft.ifft2(self.noise_fft).real

    # return self.noise
    noisex = cp.random.normal(loc=0, scale=1, size=(self.ny, self.nx))
    noisex_fft = 1j*self.kx*cp.fft.fft2(noisex)
    noisey = cp.random.normal(loc=0, scale=1, size=(self.ny, self.nx))
    noisey_fft = 1j*self.ky*cp.fft.fft2(noisey)
    self.noise_fft = noisex_fft + noisey_fft

    # normalize
    self.noise_fft[0,0] = 0

    # high (spatial domain) frequency cutoff
    self.noise_fft[self.k2 > self.parms.noise_high_freq_cutoff**2] = 0
    self.noise = cp.fft.ifft2(self.noise_fft).real

    # renormalize to eta
    self.noise *= self.parms.eta/cp.sqrt(cp.power(self.noise, 2).mean())
    self.noise_fft = cp.fft.fft2(self.noise)
    return self.noise
    
  def InitializeTimeSmoothNoise(self):
    self.noise_t = cp.zeros((self.ny, self.nx, self.parms.noise_time_smooth_steps))
    for i in range(self.parms.noise_time_smooth_steps):
      self.noise_t[:, :, i] = self.GetEtaNoise()
    
    self.omega2 = cp.fft.fftfreq(self.parms.noise_time_smooth_steps, d=1)**2  # d=dt?
      
  def GenerateTimeSmoothNoise(self):
    self.noise_t = cp.concatenate((self.noise_t[:,:,1:], cp.array(self.GetEtaNoise()).reshape(self.ny, self.nx,1)), axis=2)
    self.noise_t_hat = cp.fft.fft2(self.noise_t)

    # normalize
    self.noise_t_hat[0,0,:] = 0

    # high (temporal domain) frequency cutoff
    self.noise_t_hat[:, :, self.omega2 > self.parms.noise_time_smooth**2] = 0
    self.noise_t = cp.fft.ifft2(self.noise_t_hat).real
    self.noise = self.noise_t[:, :, -1]

    # renormalize to eta
    self.noise *= self.parms.eta/cp.sqrt(cp.power(self.noise, 2).mean())
    self.noise_fft = cp.fft.fft2(self.noise)
    return self.noise

  def InitFieldFlat(self, noisy = True):
    self.phi = self.parms.rho0 * cp.ones((self.nx, self.ny))
    self.phi += self.GetEtaNoise() if noisy & (self.parms.eta != 0) else 0
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi0 = cp.fft.ifft2(self.phi_hat).real
    self.t = 0

  def InitFieldCrystal(self, A = None, noisy = True):
    if A == None:
      A == self.parms.rho0/6
    
    q1 = cp.array([-np.sqrt(3)*0.5, -0.5])
    q2 = cp.array([0.0, 1.0])
    q3 = cp.array([np.sqrt(3)*0.5, -0.5])

    self.phi = 2 * A * (cp.cos(self.r.dot(q1)) + cp.cos(self.r.dot(q2)) + cp.cos(self.r.dot(q3))) + self.parms.rho0
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi_hat[0,0] = self.nx*self.ny*self.parms.rho0
    self.phi = cp.fft.ifft2(self.phi_hat).real
    self.phi += self.GetEtaNoise() if noisy & (self.parms.eta != 0) else 0
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi_hat[0,0] = self.nx*self.ny*self.parms.rho0
    self.phi = cp.fft.ifft2(self.phi_hat).real
    self.phi0 = self.phi.copy()
    self.t = 0

  def AddNoise(self):
    self.phi += self.GetEtaNoise()
    self.phi_hat = cp.fft.fft2(self.phi)
    self.phi0 = cp.fft.ifft2(self.phi_hat).real

  def TimeStep(self):
    # vacancy terms
    self.vac = -6*1500*cp.power(self.phi,2)*(self.phi < 0)
    self.vac_hat = cp.fft.fft2(self.vac)

    # Log terms

    # phi^3 term
    self.phi3 = cp.power(self.phi, 3)
    self.phi3_hat = cp.fft.fft2(self.phi3)

    self.phi_hat = self.phi_hat + self.parms.dt * (self.lincoeff * self.phi_hat - self.k2 * (self.phi3_hat + self.vac_hat))
    self.phi = cp.fft.ifft2(self.phi_hat).real
    self.t += self.parms.dt

  def TimeStepCross(self):
    self.phi_hat = cp.fft.fft2(self.phi)
        
    # if noise_dynamics is true, add noise to phi2 and phi3
    if self.parms.noise_dynamics:
      if self.parms.time_smooth_noise_dynamics:
        self.GenerateTimeSmoothNoise()
      else:
        self.GetEtaNoise()

    self.N0_hat = self.Get_N_hat(self.phi)

    self.phi_hat0 = self.expcoeff * self.phi_hat + -self.k2 * self.expcoeff_nonlin * self.N0_hat
    self.phi0 = cp.fft.ifft2(self.phi_hat0).real

    self.N1_hat = (self.Get_N_hat(self.phi0) - self.N0_hat)/self.parms.dt

    phi_hat1 = self.expcoeff * self.phi_hat + -self.k2 * (self.expcoeff_nonlin * self.N0_hat + self.expcoeff_nonlin2 * self.N1_hat)
    phi1 = cp.fft.ifft2(phi_hat1).real

    delta_phi = phi1 - self.phi0
    if delta_phi.max() - delta_phi.min() > 0.01:
      self.phi0 = phi1
      self.N1_hat = (self.Get_N_hat(self.phi0) - self.N0_hat)/self.parms.dt

      delta_phi = phi1 - self.phi0
      if delta_phi.max() - delta_phi.min() > 0.01:
        raise Exception("predictor-corrector failed")

    # throw an exception if phi max > 3
    if cp.max(phi1) > 5:
      raise Exception("phi max > 5")
    
    # # if noise_dynamics is true, add noise to phi2 and phi3
    # if self.parms.noise_dynamics:
    #   self.GetEtaNoise()
    #   phi_hat1 += -self.k2 * self.noise_fft
    #   phi1 = cp.fft.ifft2(phi_hat1).real

    self.phi_hat = phi_hat1
    self.phi = phi1
    self.t += self.parms.dt

  def Get_N_hat(self, phi):
    # phi^2 term
    self.phi2 = self.parms.t * cp.power(phi, 2)

    # phi^3 term
    self.phi3 = self.parms.v * cp.power(phi, 3)

    # vacancy energy (stored as member for debugging)
    self.vac = -6*self.parms.Hng*cp.power(phi,2)*(phi < 0) # 3 ϕ (|ϕ| - ϕ) = { -6ϕ^2 where ϕ < 0, or, 0 where ϕ > 0 }

    # nl energy (stored as member for debugging)
    self.ln = self.parms.Hln * cp.where(phi > -self.parms.a + np.exp(self.minLog), cp.log(phi + self.parms.a), self.minLog)

    if self.parms.Vb is not None:
      _VbMax = cp.max(self.parms.Vb)
      _x = 1 - self.parms.Vb / _VbMax
      self.phi2 *= _x
      self.phi2 *= _x
      self.vac *= _x
      self.ln *= _x
      self.boundary = self.parms.Vb * (phi - self.parms.phi_b)
    else:
      self.boundary = 0
    
    # if noise_dynamics is true, add noise to phi2 and phi3
    if self.parms.noise_dynamics:
      return cp.fft.fft2(self.phi2 + self.phi3 + self.vac + self.ln + self.boundary) + self.noise_fft
    
    return cp.fft.fft2(self.phi2 + self.phi3 + self.vac + self.ln + self.boundary)

  def CalcEnergyDensity(self):
    self.phi_hat = cp.fft.fft2(self.phi)
    self.energy_lin_1 = cp.fft.ifft2(self.linenergycoeff * self.phi_hat).real
    self.energy_lin = self.phi * self.energy_lin_1

    self.energy_ln = (self.phi + self.parms.a) * cp.where(self.phi > -self.parms.a, cp.log(self.phi + self.parms.a), 0)

    self.energy_poly = 1/2 * (self.parms.r + self.parms.b) * cp.power(self.phi,2) + 1/3 * self.parms.t * cp.power(self.phi,3) + 1/4 * self.parms.v * cp.power(self.phi,4)

    self.energy = (self.energy_lin + self.energy_ln + self.energy_poly)
    self.f = self.energy.sum() / (self.nx * self.dx * self.ny * self.dy)
    return


## Single Particle Tracking (SPT)

In [ ]:
def ComputeCM(phi: np.ndarray, time: float, dx=10, dy=10, min_distance=5, threshold_abs=0.8):
    """
    Computes the center of mass for peaks in a 2D density field.
    
    Parameters:
        phi (np.ndarray): 2D density field.
        frame_number (int): Frame number for tracking.
        dx, dy (int): Region size for CM calculation.
        min_distance (int): Minimum separation between peaks.
        threshold_abs (float): Absolute intensity threshold for peak detection.
    
    Returns:
        np.ndarray: Array of [frame_number, cm_x_global, cm_y_global, mass] for each peak.
    """
    # Detect peaks efficiently
    peaks = peak_local_max(phi, min_distance=min_distance, threshold_abs=threshold_abs, exclude_border=False)
    
    # Preallocate list
    cm_list = np.zeros((len(peaks), 4))  # [frame_number, cm_x, cm_y, mass]
    
    for i, (y_peak, x_peak) in enumerate(peaks):
        # Define bounding box
        x_start, x_end = max(0, x_peak - dx//2), min(phi.shape[1], x_peak + dx//2)
        y_start, y_end = max(0, y_peak - dy//2), min(phi.shape[0], y_peak + dy//2)

        # Extract region
        region = phi[y_start:y_end, x_start:x_end]

        # Compute CM
        cm_y, cm_x = center_of_mass(region)
        mass = region.sum()

        # Store results
        cm_list[i] = [time, y_start + cm_y, x_start + cm_x, mass]

    return cm_list

## Plotting

In [ ]:
def PlotTimeStep(_pfc):
  # vacancy terms
  _phi_hat = _pfc.phi_hat.get()
  _phi = _pfc.phi.get()
  _lincoeff = _pfc.lincoeff.get()
  _k2 = _pfc.k2.get()
  _dt = _pfc.parms.dt

  _vac = 6*1500*np.power(_phi,2)*(_phi < 0)
  _vac_hat = np.fft.fft2(_vac)

  # phi^3 term
  _phi3 = np.power(_phi, 3)
  _phi3_hat = np.fft.fft2(_phi3)

  _dphi_hat_dt = _dt * (_lincoeff * _phi_hat - _k2 * (_phi3_hat + _vac_hat))

  plt.figure(figsize=(8, 8))

  plt.subplot(4, 2, 1)
  plt.title("$\phi$")
  plt.imshow(_phi, cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 2)
  plt.title("$\hat{\phi}$")
  plt.contourf(np.abs(_phi_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 3)
  plt.title("vac")
  plt.contourf(np.abs(_vac), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 4)
  plt.title("$\hat{vac}$")
  plt.contourf(np.abs(_vac_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 5)
  plt.title("$\phi^3$")
  plt.contourf(np.abs(_phi3), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 6)
  plt.title("$\hat{\phi}^3$")
  plt.contourf(np.abs(_phi3_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  # self.phi_hat = self.phi_hat + self.parms.dt * (self.lincoeff * self.phi_hat - self.k2 * (self.phi3_hat + self.vac_hat))
  plt.subplot(4, 2, 7)
  plt.title("$t1$")
  plt.contourf(np.abs(_lincoeff * _phi_hat), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.subplot(4, 2, 8)
  plt.title("$t2$")
  plt.contourf(np.abs(_k2 * (_phi3_hat + _vac_hat)), _pfc.nx.get(), cmap='viridis')
  plt.colorbar()

  plt.tight_layout()
  plt.show()

In [ ]:
def PlotChemicalPotential(_phi, kx, ky, k2, k4, t, epsilon, g, beta, threshold_abs=1.8):
  # find center of mass
  _cm_data = ComputeCM(_phi, t, threshold_abs=threshold_abs)

  # calcultate the components of the chemical potential
  _mu_ln = np.log(_phi)
  _mu_epsilon = epsilon * _phi
  _mu_g = g * _phi**2
  _mu_v0 = _phi**3
  _mu_beta = (beta * cp.fft.ifft2((cp.array(k4) - 2 * cp.array(k2) + 1) * cp.fft.fft2(cp.array(_phi)))).get().real
  _mu = _mu_ln + _mu_beta + _mu_epsilon + _mu_g + _mu_v0

  # calculate time derivative of density in Fourier space
  _dphi_hat = -k2 * np.fft.fft2(_mu)
  _dphi = np.fft.ifft2(_dphi_hat).real

  # calculate the density current
  _dphi_dx_hat = 1j*kx*_dphi_hat
  _dphi_dy_hat = 1j*ky*_dphi_hat
  _dphi_dx = np.fft.ifft2(_dphi_dx_hat).real
  _dphi_dy = np.fft.ifft2(_dphi_dy_hat).real

  # calculate a low frequency time derivative of density
  _dphi_hat_lf = _dphi_hat.copy()
  _dphi_hat_lf[k2 > 0.8**2] = 0
  _dphi_lf = np.fft.ifft2(_dphi_hat_lf).real

  # calculate the low frequency density current
  _dphi_dx_hat_lf = 1j*kx*_dphi_hat_lf
  _dphi_dy_hat_lf = 1j*ky*_dphi_hat_lf
  _dphi_dx_lf = np.fft.ifft2(_dphi_dx_hat_lf).real
  _dphi_dy_lf = np.fft.ifft2(_dphi_dy_hat_lf).real

  # normalize vectors:
  _dphi_lf_max = np.sqrt(_dphi_dx_lf**2 + _dphi_dy_lf**2).max()
  _dphi_dx_lf /= _dphi_lf_max
  _dphi_dy_lf /= _dphi_lf_max

  layout = dict(hoversubplots='axis', title='mu epsilon', grid=dict(rows=1, columns=2),
                yaxis=dict(scaleanchor="x"),
                width=__pfc.nx*4 + 100,
                height=__pfc.ny*2 + 100,
                margin=dict(l=1, r=30, t=1, b=1),
                xaxis_showgrid=False,
                yaxis_showgrid=False,
                xaxis_zeroline=False,
                yaxis_zeroline=False,
                xaxis_visible=False,
                yaxis_visible=False,
                plot_bgcolor='rgba(0,0,0,0)',)
  
  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_mu_ln, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"mu ln", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_mu_epsilon, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"mu epsilon", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_mu_g, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"mu g", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_mu_v0, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"mu v0", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_mu_beta, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"mu beta", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_mu, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"mu", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = go.Figure(layout=layout)
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_dphi, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(
      title=dict(text=f"dphi", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  fig = ff.create_quiver(np.meshgrid(np.arange(_phi.shape[1])[::5],np.arange(180)[::5])[0],
                         np.meshgrid(np.arange(_phi.shape[0])[::5],np.arange(180)[::5])[1],
                         _dphi_dx_lf[::5,::5], _dphi_dy_lf[::5,::5],
                       scale=3,
                       arrow_scale=.4,
                       name='quiver',
                       line_width=1,
                       line_color='cyan',
                       xaxis='x')
  fig.add_trace(go.Heatmap(z=_phi, colorscale='hot', xaxis='x', showscale=False))
  fig.add_trace(go.Heatmap(z=_dphi_lf, colorscale='Hot', xaxis='x2', colorbar=dict(thickness=10)))
  fig.update_layout(layout)
  fig.update_layout(
      title=dict(text=f"dphi Low Freq.", font=dict(size=20), x=0.1, y=0.95, yref='paper')
  )
  fig.show()

  return _dphi

In [ ]:
_phi.shape

## Saving / Restoring

In [ ]:
def SaveData(_pfc, _filename):
  np.savez(_filename + f'_N!{_pfc.parms.N}_rho0!{_pfc.parms.rho0:0.2f}_seed!{_pfc.parms.seed}_t!{_pfc.t:0.2f}', parms=_pfc.parms.__dict__, t=_pfc.t, phi=_pfc.phi.get())

In [ ]:
def ReadPFCFile(_filename):
  data = np.load(_filename, allow_pickle=True)
  pfc = PFC2D_Vacancy()
  pfc.parms.__dict__ = data['parms'].item()
  pfc.InitParms()
  pfc.phi = cp.array(data['phi'])
  pfc.phi_hat = cp.fft.fft2(pfc.phi)
  pfc.phi0 = cp.fft.ifft2(pfc.phi_hat).real
  pfc.t = data['t']
  return pfc

In [ ]:
def AppendCM(master_list, filepath="cm.npz"):
    """
    Appends new frames from master_list to an HDF5 file.

    Parameters:
        master_list (np.ndarray): Array of [frame_number, cm_x, cm_y, mass] to append.
        filepath (str): Path to the HDF5 file (defaults to 'cm.npz').

    Behavior:
        - Creates the dataset if it doesn't exist.
        - Appends only new frames, avoiding overwrite.
        - Clears master_list after writing to prevent memory growth.
    """
    with h5py.File(filepath, "a") as f:
        if "cm_data" not in f:
            # Create dataset with maxshape for unlimited rows
            dset = f.create_dataset("cm_data", data=master_list, maxshape=(None, 4), dtype="float32")
        else:
            dset = f["cm_data"]
            prev_shape = dset.shape[0]
            dset.resize((prev_shape + master_list.shape[0], 4))
            dset[prev_shape:] = master_list

    return master_list  # Ensure it's empty after writing

## Testing

### Phase diagram

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3
__pfc.parms.a = 0
__pfc.parms.b = 2
__pfc.parms.t = -1
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.5

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 360
__pfc.parms.PPU = 30
__pfc.parms.eta = 0.01
__pfc.parms.noise_dynamics = False
__pfc.parms.dt = 0.01
__pfc.parms.seed = 90
__pfc.parms.Vb = None
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
# __pfc.SetGeometry(180, 180, 10, 1)
# __pfc.InitFieldFlat(noisy = True)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = True)
__pfc.phi[:,:78] = 0.216
__pfc.phi[:,-78:] = 0.216
__pfc.phi[:67,:] = 0.216
__pfc.phi[-67:,:] = 0.216

# __pfc.phi *= 1 - __pfc.parms.Vb / __pfc.parms.Vb.max()
# __pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
# __pfc.phi[81:90,85:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

__total_rt = int(6 * 1e5)
for i in range(__total_rt):
  try:
    if i % int(__total_rt/30) == int(__total_rt/30) - 1:
      __pfc.CalcEnergyDensity()
      __cm_data = ComputeCM(__pfc.phi.get(), __pfc.t, dx=30, dy=30, threshold_abs=0.8)
      __area = (__cm_data[:,1].max() - __cm_data[:,1].min()) * (__cm_data[:,2].max() - __cm_data[:,2].min())
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}, area: {__area}')

    if i % int(__total_rt/3) == int(__total_rt/3) - 1:
        fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
        fig.update_layout(
            yaxis=dict(scaleanchor="x"),
            width=__pfc.nx*2 + 100,
            height=__pfc.ny*2 + 100,
            margin=dict(l=0, r=0, t=100, b=0),
            xaxis_showgrid=False,
            yaxis_showgrid=False,
            xaxis_zeroline=False,
            yaxis_zeroline=False,
            xaxis_visible=False,
            yaxis_visible=False,
            plot_bgcolor='rgba(0,0,0,0)',
            title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
        )

        # scatter plot markers of the peaks
        __x_coords, __y_coords = __cm_data[:, 2], __cm_data[:, 1]
        fig.add_trace(go.Scatter(
            x=__x_coords,
            y=__y_coords,
            mode='markers',
            marker=dict(symbol='x', color='green', size=5),
            name="Peaks"
        ))

        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
__r_ll = __cm_data[((__cm_data[:,2] - 72)**2 + (__cm_data[:,1] - 72)**2 < 10**2)][0, 1:3]*np.array([__pfc.dy, __pfc.dx])
__r_lr = __cm_data[((__cm_data[:,2] - 260)**2 + (__cm_data[:,1] - 72)**2 < 10**2)][0, 1:3]*np.array([__pfc.dy, __pfc.dx])
__r_ul = __cm_data[((__cm_data[:,2] - 72)**2 + (__cm_data[:,1] - 288)**2 < 10**2)][0, 1:3]*np.array([__pfc.dy, __pfc.dx])
__r_ur = __cm_data[((__cm_data[:,2] - 260)**2 + (__cm_data[:,1] - 288)**2 < 10**2)][0, 1:3]*np.array([__pfc.dy, __pfc.dx])
__r_ll, __r_lr, __r_ul, __r_ur

In [ ]:
np.array([72.0476354, 71.6673911])*np.array([2,-2])

In [ ]:
# calculate the angle from lower left to lower right
__angleB = np.arctan2(__r_lr[0] - __r_ll[0], __r_lr[1] - __r_ll[1])
__angleB

In [ ]:
__angleB = 0

__xB = (__r_ll[1] + np.cos(__angleB) * np.linspace(-10, 80, 1000))
__yB = (__r_ll[0] + np.sin(__angleB) * np.linspace(-10, 80, 1000))

# using 2D interpolation on __pfc.phi, calculate phi(x) along the bottom
from scipy.interpolate import RegularGridInterpolator

x = __pfc.dx * np.arange(__pfc.nx)
y = __pfc.dy * np.arange(__pfc.ny)

interpolator = RegularGridInterpolator((x, y), __pfc.phi.get(), bounds_error=False, fill_value = 0)
__phiB = interpolator(np.array([__yB % (__pfc.dy * __pfc.ny), __xB % (__pfc.dx * __pfc.nx)]).T)
print(__phiB.min(), __phiB.max(), (__phiB.min() + __phiB.max())/2)
# plot

__A0p = 0.8628
__qp = 0.8927

fig = go.Figure(data=[
    go.Scatter(x = __xB, y = __phiB),
    go.Scatter(x = __xB[(__xB > 0) & (__xB < 4 * np.pi / np.sqrt(3) / __qp)], y = 2 * __A0p + 4 * __A0p * np.cos(__xB * np.sqrt(3)/2 * __qp))])
fig.show()

In [ ]:
y

# Model

### Rescaled coefficients

In [ ]:
pfc = PFC2D_Vacancy()
pfc.parms.r = -1.6*3.1
pfc.parms.a = 1/np.e - .2
pfc.parms.b = 1.0*3.1
pfc.parms.t = -0.3*3.1
pfc.parms.v = 0.9*3.1
pfc.parms.Hng = 0
pfc.parms.Hln = 1
pfc.parms.N = 100
pfc.parms.PPU = 10
pfc.parms.rho0 = 0.1
pfc.parms.eta = 0.2
pfc.parms.dt = 8e-3
pfc.parms.seed = 1

pfc.InitParms()
pfc.InitFieldFlat()

# show the image scaled to 1 px per point
for i in range(20000):
  try:
    # if i % 100 == 0:
    #   pfc.AddNoise()
    pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(pfc.parms.__dict__)
print(pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=pfc.parms.N*2,
    height=pfc.parms.N*2,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

# SaveData(pfc, 'ln_flat')

In [ ]:
pfc = PFC2D_Vacancy()
_r = -1.6*3.1
_b = 1.0*3.1
_t = -0.3*3.1
_v = 0.9*3.1
_a = 1/np.e - .2
_rho0 = 0.1

pfc.parms.a = 0
pfc.parms.r = _r - 2*_t*_a + 3*_v*_a**2
pfc.parms.b = _b
pfc.parms.t = _t - 3*_v*_a
pfc.parms.v = _v
pfc.parms.Hng = 0
pfc.parms.Hln = 1
pfc.parms.N = 100
pfc.parms.PPU = 10
pfc.parms.rho0 = _rho0 + _a
pfc.parms.eta = 0.2
pfc.parms.dt = 8e-3
pfc.parms.seed = 1

pfc.InitParms()
pfc.InitFieldFlat()

# show the image scaled to 1 px per point
for i in range(20000):
  try:
    # if i % 100 == 0:
    #   pfc.AddNoise()
    pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(pfc.parms.__dict__)
print(pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=pfc.parms.N*2,
    height=pfc.parms.N*2,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

# SaveData(pfc, 'ln_flat')

In [ ]:
pfc = PFC2D_Vacancy()
_r = -1.6*3.1
_b = 1.0*3.1
_t = -0.3*3.1
_v = 0.9*3.1
_a = 1/np.e - .2
_rho0 = 0.1

pfc.parms.a = 0

__r = _r - 2*_t*_a + 3*_v*_a**2
__b = _b
__t = _t - 3*_v*_a
__v = _v
__rho0 = _rho0 + _a

pfc.parms.r = __r*__v**(-1/3)
pfc.parms.b = __b*__v**(-1/3)
pfc.parms.t = __t*__v**(-2/3)
pfc.parms.v = 1
pfc.parms.rho0 = __rho0*__v**(1/3)


pfc.parms.Hng = 0
pfc.parms.Hln = 1
pfc.parms.N = 100
pfc.parms.PPU = 10
pfc.parms.eta = 0.2
pfc.parms.dt = 8e-3*__v**(1/3)
pfc.parms.seed = 1

pfc.InitParms()
pfc.InitFieldFlat()

# show the image scaled to 1 px per point
for i in range(20000):
  try:
    # if i % 100 == 0:
    #   pfc.AddNoise()
    pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(pfc.parms.__dict__)
print(pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=pfc.parms.N*2,
    height=pfc.parms.N*2,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

# SaveData(pfc, 'ln_flat')

### Maximum density

In [ ]:
pfc = PFC2D_Vacancy()
pfc.parms.r = -1.6*3
pfc.parms.a = 1/np.e - .2
pfc.parms.b = 1.0*3
pfc.parms.t = -0.3*3
pfc.parms.v = 0.9*3
pfc.parms.Hng = 0*3
pfc.parms.Hln = 0.33333*3
pfc.parms.N = 256
pfc.parms.PPU = 10
pfc.parms.rho0 = 0.20
pfc.parms.eta = 0.2
pfc.parms.dt = 8e-3
pfc.parms.seed = 1

pfc.InitParms()
pfc.InitFieldFlat()

# show the image scaled to 1 px per point
for i in range(5000):
  try:
    # if i % 100 == 0:
    #   pfc.AddNoise()
    pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=pfc.parms.N*2,
    height=pfc.parms.N*2,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

# SaveData(pfc, 'ln_flat')

In [ ]:
pfc = PFC2D_Vacancy()
_r = -1.6*3.1
_b = 1.0*3.1
_t = -0.3*3.1
_v = 0.9*3.1
_a = 1/np.e - .2
_rho0 = 0.2

pfc.parms.a = 0

__r = _r - 2*_t*_a + 3*_v*_a**2
__b = _b
__t = _t - 3*_v*_a
__v = _v
__rho0 = _rho0 + _a

pfc.parms.r = __r*__v**(-1/3)
pfc.parms.b = __b*__v**(-1/3)
pfc.parms.t = __t*__v**(-2/3)
pfc.parms.v = 1
pfc.parms.rho0 = __rho0*__v**(1/3)


pfc.parms.Hng = 0
pfc.parms.Hln = 1
pfc.parms.N = 256
pfc.parms.PPU = 10
pfc.parms.eta = 0.2
pfc.parms.dt = 8e-3*__v**(1/3)
pfc.parms.seed = 1

pfc.InitParms()
pfc.InitFieldFlat()

# show the image scaled to 1 px per point
for i in range(10000):
  try:
    if i % 10000 == 0:
       pfc.CalcEnergyDensity()
       print(f'f: {pfc.f:.10f}')
    pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(pfc.parms.__dict__)
print(pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=pfc.parms.N*2,
    height=pfc.parms.N*2,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

# SaveData(pfc, 'ln_flat')

In [ ]:
# make a histogram of the pfc.phi values
fig = go.Figure(data=go.Histogram(histnorm="probability",x=pfc.phi.flatten().get()))

fig.show()

In [ ]:
_pfc = PFC2D_Vacancy()
_pfc.parms.r = -3.1339050579032284
_pfc.parms.a = 0
_pfc.parms.b = 2.2020489023671126
_pfc.parms.t = -1.1782718056073347
_pfc.parms.v = 1
_pfc.parms.rho0 = 0.5178932522368416

_pfc.parms.Hng = 0
_pfc.parms.Hln = 1
_pfc.parms.N = 180
_pfc.parms.PPU = 10
_pfc.parms.eta = 0.2
_pfc.parms.dt = 0.011262238533095706
_pfc.parms.seed = 1

_pfc.InitParms()
_pfc.InitFieldCrystal(A = (1.92 - pfc.parms.rho0)/6, noisy = False)

# make a histogram of the pfc.phi values
fig = go.Figure(data=go.Histogram(histfunc='count',x=_pfc.phi.flatten().get()))

fig.show()

In [ ]:
_pfc = PFC2D_Vacancy()
_pfc.parms.r = -3.1339050579032284
_pfc.parms.a = 0
_pfc.parms.b = 2.2020489023671126
_pfc.parms.t = -1.1782718056073347
_pfc.parms.v = 1
_pfc.parms.rho0 = 0.5178932522368416

_pfc.parms.Hng = 0
_pfc.parms.Hln = 1
_pfc.parms.N = 180
_pfc.parms.PPU = 10
_pfc.parms.eta = 0.2
_pfc.parms.dt = 0.011262238533095706
_pfc.parms.seed = 1

_pfc.InitParms()
_pfc.InitFieldCrystal(A = (1.92 - pfc.parms.rho0)/6, noisy = False)

# show the image scaled to 1 px per point
_pfc.SetDT(_pfc.parms.dt*1e-7)
for s in range(6):
    for i in range(1000):
        try:
            _pfc.TimeStepCross()
        except Exception as e:
            print(s, i, _pfc.parms.dt, e)
            break
    _pfc.CalcEnergyDensity()
    print(f'f: {_pfc.f:.10f}')
    _pfc.SetDT(_pfc.parms.dt*10)

_pfc.SetDT(0.011262238533095706)
for i in range(10000):
  try:
    if i % 10000 == 0:
       _pfc.CalcEnergyDensity()
       print(f'f: {_pfc.f:.10f}')
    _pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(_pfc.parms.__dict__)
print(_pfc.t, _pfc.phi.mean(), _pfc.phi.min(), _pfc.phi.max(), np.power((_pfc.phi - _pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=_pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_pfc.parms.N*2,
    height=_pfc.parms.N*2,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()


In [ ]:

# make a histogram of the pfc.phi values
fig = go.Figure(data=go.Histogram(histfunc='count',x=_pfc.phi.flatten().get()))

fig.show()

### Examples

#### 1. Noise

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.0
__pfc.parms.t = -1.0
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.31

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 100
# __pfc.parms.time_smooth_noise_dynamics = True
# __pfc.parms.noise_time_smooth_steps = 60
# __pfc.parms.noise_time_smooth = 0.001

__pfc.InitParms()
# __pfc.SetGeometry(40, 36, 10, 1)
__pfc.InitFieldFlat(noisy = True)
__pfc.InitializeTimeSmoothNoise()

# raise SystemExit("Stopping execution")

outputPath = f'05-07-2025/noise_T0/'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(1):
  try:
    # if i % 1000 == 999:
    # __pfc.CalcEnergyDensity()
    # print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
    
    fig = go.Figure(data=go.Heatmap(z=__pfc.noise.get(), colorscale='hot', colorbar_thickness=10, zmin=-0.4, zmax=0.4))
    fig.update_layout(
        yaxis=dict(scaleanchor="x"),
        width=__pfc.nx*2 + 100,
        height=__pfc.ny*2 + 100,
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        xaxis_zeroline=False,
        yaxis_zeroline=False,
        xaxis_visible=False,
        yaxis_visible=False,
        plot_bgcolor='rgba(0,0,0,0)'
    )

    fig.write_image(os.path.join(outputPath, f'noise_K{__pfc.parms.noise_high_freq_cutoff:02}.png'))
    # if i % 10000 == 9999:
    fig.show()

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    # __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
# make a histogram of the __pfc.noise in plotly go
import plotly.graph_objects as go

noisex = cp.random.normal(loc=0, scale=1, size=(__pfc.ny, __pfc.nx))
noisex_fft = 1j*__pfc.kx*cp.fft.fft2(noisex)
noisey = cp.random.normal(loc=0, scale=1, size=(__pfc.ny, __pfc.nx))
noisey_fft = 1j*__pfc.ky*cp.fft.fft2(noisey)
noise_fft = noisex_fft + noisey_fft

fig = go.Figure(data=[go.Histogram(x=noisex.get().ravel(), nbinsx=100),
                      go.Histogram(x=noisey.get().ravel(), nbinsx=100)])
fig.update_layout(title='Histogram of Noise Values', xaxis_title='Noise Value', yaxis_title='Frequency')
fig.show()

fig = go.Figure(data=[go.Histogram(x=noise_fft.get().real.ravel(), nbinsx=100),])
fig.update_layout(title='Histogram of Noise Values', xaxis_title='Noise Value', yaxis_title='Frequency')
fig.show()

fig = go.Figure(data=[go.Histogram(x=cp.fft.fft2(noisex).get().real.ravel(), nbinsx=100),])
fig.update_layout(title='Histogram of Noise Values', xaxis_title='Noise Value', yaxis_title='Frequency')
fig.show()

noise_a = cp.fft.ifft2(__pfc.noise_fft)
fig = go.Figure(data=[go.Histogram(x=noise_a.get().real.ravel(), nbinsx=100),])
fig.update_layout(title='Histogram of Noise Values', xaxis_title='Noise Value', yaxis_title='Frequency')
fig.show()


In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.2
__pfc.parms.t = -1.2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.52

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.15
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitializeTimeSmoothNoise()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi[140:149,126:135] = 0.00 # Vacancy
__pfc.phi_fft = cp.fft.fft2(__pfc.phi)
__pfc.phi_fft[0,0] = __pfc.parms.rho0 * __pfc.nx * __pfc.ny
__pfc.phi = cp.fft.ifft2(__pfc.phi_fft).real

__pfc.InitializeTimeSmoothNoise()

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
# fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
# if i % 10000 == 9999:
fig.show()
__pfc.CalcEnergyDensity()
print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

# raise SystemExit("Stopping execution")

# outputPath = f'noise_T15_sigma{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
# if not os.path.exists(outputPath):
#   os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 1000 == 0:
        __pfc.CalcEnergyDensity()
        print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
        
        fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10, zmin=0, zmax=1.9))
        fig.update_layout(
            yaxis=dict(scaleanchor="x"),
            width=__pfc.nx*2 + 100,
            height=__pfc.ny*2 + 100,
            margin=dict(l=0, r=0, t=0, b=0),
            xaxis_showgrid=False,
            yaxis_showgrid=False,
            xaxis_zeroline=False,
            yaxis_zeroline=False,
            xaxis_visible=False,
            yaxis_visible=False,
            plot_bgcolor='rgba(0,0,0,0)',
            title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
        )
    # fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
    if i % 10000 == 9999:
        fig.show()

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.2
__pfc.parms.t = -1.2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.50

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.05
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitializeTimeSmoothNoise()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi[140:149,126:135] = 0.00 # Vacancy
# __pfc.phi_fft = cp.fft.fft2(__pfc.phi)
# __pfc.phi_fft[0,0] = __pfc.parms.rho0 * __pfc.nx * __pfc.ny
# __pfc.phi = cp.fft.ifft2(__pfc.phi_fft).real

__pfc.InitializeTimeSmoothNoise()

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
# fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
# if i % 10000 == 9999:
fig.show()
__pfc.CalcEnergyDensity()
print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

# raise SystemExit("Stopping execution")

# outputPath = f'noise_T15_sigma{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
# if not os.path.exists(outputPath):
#   os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 1000 == 0:
        __pfc.CalcEnergyDensity()
        print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
        
        fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10, zmin=0, zmax=1.9))
        fig.update_layout(
            yaxis=dict(scaleanchor="x"),
            width=__pfc.nx*2 + 100,
            height=__pfc.ny*2 + 100,
            margin=dict(l=0, r=0, t=0, b=0),
            xaxis_showgrid=False,
            yaxis_showgrid=False,
            xaxis_zeroline=False,
            yaxis_zeroline=False,
            xaxis_visible=False,
            yaxis_visible=False,
            plot_bgcolor='rgba(0,0,0,0)',
            title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
        )
    # fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
    if i % 10000 == 9999:
        fig.show()

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.energy.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
# fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
# if i % 10000 == 9999:
fig.show()

In [ ]:

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
# fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
# if i % 10000 == 9999:
fig.show()

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.0
__pfc.parms.t = -1.0
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.31

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
# __pfc.SetGeometry(40, 36, 10, 1)
__pfc.InitFieldFlat(noisy = True)
__pfc.InitializeTimeSmoothNoise()

# raise SystemExit("Stopping execution")

outputPath = f'noise_T15_sigma{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(1000):
  try:
    # if i % 1000 == 999:
    __pfc.CalcEnergyDensity()
    print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
    
    fig = go.Figure(data=go.Heatmap(z=__pfc.noise.get(), colorscale='hot', colorbar_thickness=10, zmin=-0.4, zmax=0.4))
    fig.update_layout(
        yaxis=dict(scaleanchor="x"),
        width=__pfc.nx*2 + 100,
        height=__pfc.ny*2 + 100,
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        xaxis_zeroline=False,
        yaxis_zeroline=False,
        xaxis_visible=False,
        yaxis_visible=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
    )
    fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
    # if i % 10000 == 9999:
    # fig.show()

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
__pfc.omega2

#### 2. Random field

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.0
__pfc.parms.t = -1.0
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.31

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 10
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 60
__pfc.parms.noise_time_smooth = 0.001

__pfc.InitParms()
# __pfc.SetGeometry(40, 36, 10, 1)
__pfc.InitFieldFlat(noisy = True)
__pfc.InitializeTimeSmoothNoise()

# raise SystemExit("Stopping execution")

outputPath = f'diffusionTS_long_eta{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(1000000):
  try:
    if i % 1000 == 999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 10000 == 9999:
        fig.show()

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


#### 3. Perfect w/ 1 Vacancy

##### a. T0 noise

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.2
__pfc.parms.t = -1.2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.49

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.05
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = False
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitializeTimeSmoothNoise()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
# __pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[142:150,131:140] = 0.00 # Vacancy

# raise SystemExit("Stopping execution")

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
fig.show()

# outputPath = f'diffusion_long_eta{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
# if not os.path.exists(outputPath):
#   os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 1000 == 999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 10000 == 9999:
        fig.show()

      fig = go.Figure(data=go.Heatmap(z=__pfc.noise.get(), colorscale='hot', colorbar_thickness=10, zmin=-0.4, zmax=0.4))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'noise{i//1000:04}.png'))


    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


##### b. T15 noise

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.2
__pfc.parms.t = -1.2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.49

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.05
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitializeTimeSmoothNoise()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
# __pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[142:150,131:140] = 0.00 # Vacancy

# raise SystemExit("Stopping execution")

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
fig.show()

outputPath = f'diffusionTS_long_eta{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(1000000):
  try:
    if i % 1000 == 999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 10000 == 9999:
        fig.show()

      fig = go.Figure(data=go.Heatmap(z=__pfc.noise.get(), colorscale='hot', colorbar_thickness=10, zmin=-0.4, zmax=0.4))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'noise{i//1000:04}.png'))


    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


#### 4. Perfect w/ n Vacancy(ies)

##### 1 vacancy

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.0
__pfc.parms.t = -1.2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.49

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitializeTimeSmoothNoise()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
# __pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[142:150,131:140] = 0.00 # vacancy
__pfc.phi[:11,:] = 0.20 # liquid
__pfc.phi[162:,:] = 0.20 # liquid
__pfc.phi[:,:28] = 0.20 # liquid
__pfc.phi[:,166:] = 0.20 # liquid

# raise SystemExit("Stopping execution")

outputPath = f'./05-03-2025/diffusionTS_SL_long_beta{__pfc.parms.b}_eta{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)

master_list = np.empty((0, 4))  # Initialize empty array

for i in range(1000000):
  try:
    cm_data = ComputeCM(__pfc.phi.get(), __pfc.t)
    master_list = np.vstack((master_list, cm_data))  # Append new data to the array
    if i % 1000 == 999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 100000 == 99999:
        fig.show()

      fig = go.Figure(data=go.Heatmap(z=__pfc.noise.get(), colorscale='hot', colorbar_thickness=10, zmin=-0.4, zmax=0.4))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'noise{i//1000:04}.png'))

      if i % 100 == 99:
        AppendCM(master_list, os.path.join(outputPath, "cm.npz"))
        master_list = np.empty((0, 4))

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
master_list

In [ ]:
with h5py.File(os.path.join(outputPath, "cm.npz"), "r") as f:
        if "cm_data" in f:
            cm_data = np.array(f["cm_data"])
            print(cm_data.shape)
        else:
            print("Dataset 'cm_data' not found in file.")


In [ ]:
cm_data

In [ ]:
%timeit __pfc.TimeStepCross()

In [ ]:
%timeit ComputeCM(__pfc.phi.get(), __pfc.t)

In [ ]:
_phi = __pfc.phi.get()
%timeit peak_local_max(_phi, min_distance=5, threshold_abs=0.8, exclude_border = False)

In [ ]:
# Define region size
dx, dy = 10, 10

# Compute CM for each peak
cm_list = []
peaks = peak_local_max(__pfc.phi.get(), min_distance=5, threshold_abs=0.8, exclude_border=False)
print(peaks, peaks.shape)
for y_peak, x_peak in peaks:
    # Extract dx × dy region around peak
    x_start, x_end = max(0, x_peak - dx//2), min(_phi.shape[1], x_peak + dx//2)
    y_start, y_end = max(0, y_peak - dy//2), min(_phi.shape[0], y_peak + dy//2)
    
    region = _phi[y_start:y_end, x_start:x_end]
    
    # Compute CM using scipy's center_of_mass
    cm_y, cm_x = center_of_mass(region)
    mass = region.sum()
    
    # Adjust CM to match global coordinates
    cm_x_global = x_start + cm_x
    cm_y_global = y_start + cm_y
    
    cm_list.append([cm_x_global, cm_y_global, mass])

# Print results
cm_list = np.array(cm_list)
print(cm_list)

In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
# scatter plot markers of the peaks
x_coords, y_coords = cm_list[:, 0], cm_list[:, 1]
fig.add_trace(go.Scatter(
    x=x_coords,
    y=y_coords,
    mode='markers',
    marker=dict(symbol='x', color='green', size=5),
    name="Peaks"
))
fig.show()

#### 5. Search for honeycomb

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 8
__pfc.parms.t = -1.6 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 1.5 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 30
__pfc.parms.eta = 0.01
__pfc.parms.noise_dynamics = False
__pfc.parms.dt = 0.01
__pfc.parms.seed = 90
__pfc.parms.Vb = None
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
# __pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitFieldFlat(noisy = True)
# __pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
# __pfc.phi *= 1 - __pfc.parms.Vb / __pfc.parms.Vb.max()
# __pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
# __pfc.phi[81:90,85:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

# show the image scaled to 1 px per point
for i in range(6000):
  try:
    if i % 200== 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
      if i % 2000 == 1999:
        fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
        fig.update_layout(
            yaxis=dict(scaleanchor="x"),
            width=__pfc.nx*2 + 100,
            height=__pfc.ny*2 + 100,
            margin=dict(l=0, r=0, t=100, b=0),
            xaxis_showgrid=False,
            yaxis_showgrid=False,
            xaxis_zeroline=False,
            yaxis_zeroline=False,
            xaxis_visible=False,
            yaxis_visible=False,
            plot_bgcolor='rgba(0,0,0,0)',
            title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
        )

        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

#### misc

In [ ]:
# make a histogram of __pfc.noise
fig = go.Figure(data=go.Histogram(histfunc='count',x=__pfc.noise.flatten().get()))
fig.update_layout(
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.show()

In [ ]:
__pfc.k2 > 1

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.0
__pfc.parms.t = -1.0
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.31

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 5.3
__pfc.parms.dt = 0.01
__pfc.parms.seed = 1001
__pfc.parms.noise_dynamics = True

__pfc.InitParms()
# __pfc.SetGeometry(40, 36, 10, 1)
__pfc.InitFieldFlat(noisy = True)

outputPath = f'diffusion_long_eta{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(1000000):
  try:
    if i % 1000 == 999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 10000 == 9999:
        fig.show()

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break



In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2020489023671126
__pfc.parms.t = -1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.2
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 1

__pfc.InitParms()
__pfc.SetGeometry(40, 36, 10, 0.9)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 10000 == 0:
       __pfc.CalcEnergyDensity()
       print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}')
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

print(__pfc.parms.__dict__)
print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()


In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2020489023671126
__pfc.parms.t = -1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.46 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.2
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 1

__pfc.InitParms()
__pfc.SetGeometry(40, 36, 10, 1)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi[14:23,16:25] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 10000 == 0:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)'
      )
      fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2020489023671126
__pfc.parms.t = -1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.46 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.2
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 1

__pfc.InitParms()
__pfc.SetGeometry(40, 36, 10, 0.96)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi[14:23,16:25] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

# show the image scaled to 1 px per point
for i in range(10000):
  try:
    if i % 1000 == 0:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)'
      )
      fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
__pfc.CalcEnergyDensity()

In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*4 + 100,
    height=__pfc.ny*4 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.energy_lin.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*4 + 100,
    height=__pfc.ny*4 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.energy_ln.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*4 + 100,
    height=__pfc.ny*4 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.energy_poly.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*4 + 100,
    height=__pfc.ny*4 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2020489023671126
__pfc.parms.t = -1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 99

__pfc.InitParms()
__pfc.SetGeometry(40, 36, 10, 0.96)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = True)
__pfc.phi[14:23,16:25] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

outputPath = f'diffusion_lowtemp{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 100 == 99:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.show()
      fig.write_image(os.path.join(outputPath, f'diffusion{i//100:04}.png'))

    if i % 5 == 2:
      __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2020489023671126
__pfc.parms.t = -1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.50 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 150

__pfc.InitParms()
__pfc.SetGeometry(80, 72, 10, 1)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = True)
__pfc.phi[32:41,36:45] = 0.00
# __pfc.noiseMask[:10,:10] = 0
# __pfc.noiseMask[-10:,:10] = 0
# __pfc.noiseMask[:10,-10:] = 0
# __pfc.noiseMask[-10:,-10:] = 0

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

outputPath = f'diffusion_long_medtemp_newnoise{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(600000):
  try:
    if i % 200 == 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}", font=dict(size=12), x=0.05, y=0.95, yref='paper')
      )
      if i % 20000 == 19999:
        fig.show()
      fig.write_image(os.path.join(outputPath, f'diffusion{i//200:04}.png'))

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2020489023671126
__pfc.parms.t = -1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.4275 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 1.25
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 150

__pfc.InitParms()
__pfc.SetGeometry(80, 72, 10, 1)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi[32:41,36:45] = 0.00
# __pfc.noiseMask[:10,:10] = 0
# __pfc.noiseMask[-10:,:10] = 0
# __pfc.noiseMask[:10,-10:] = 0
# __pfc.noiseMask[-10:,-10:] = 0

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

outputPath = f'diffusion_med_rho0!{__pfc.parms.rho0}_eta!{__pfc.parms.eta:0.3f}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(60000):
  try:
    if i % 200 == 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}", font=dict(size=12), x=0.05, y=0.95, yref='paper')
      )
      if i % 20000 == 19999:
        fig.show()
      fig.write_image(os.path.join(outputPath, f'diffusion{i//200:04}.png'))

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
__pfc.x

In [ ]:
fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()
fig.write_image("test.png")

#### Energy / Chemical Potential Visualization

##### Perfect lattice - annealed

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.0
__pfc.parms.a = 0
__pfc.parms.b = 2.0
__pfc.parms.t = -1.2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.49

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.dt = 0.01
__pfc.parms.seed = 13001
__pfc.parms.noise_dynamics = True
__pfc.parms.noise_high_freq_cutoff = 0.8
__pfc.parms.time_smooth_noise_dynamics = True
__pfc.parms.noise_time_smooth_steps = 15
__pfc.parms.noise_time_smooth = 0.01

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
__pfc.InitializeTimeSmoothNoise()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
# __pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
# __pfc.phi[142:150,131:140] = 0.00 # vacancy
# __pfc.phi[:11,:] = 0.20 # liquid
# __pfc.phi[162:,:] = 0.20 # liquid
# __pfc.phi[:,:28] = 0.20 # liquid
# __pfc.phi[:,166:] = 0.20 # liquid

# raise SystemExit("Stopping execution")

# outputPath = f'./05-03-2025/diffusionTS_SL_long_beta{__pfc.parms.b}_eta{__pfc.parms.eta}_seed{__pfc.parms.seed:03}'
# if not os.path.exists(outputPath):
#   os.mkdir(outputPath)

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)

for i in range(100000):
  try:
    if i % 10000 == 9999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 100000 == 99999:
        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

## turn off noise
__pfc.parms.eta = 0.0
__pfc.parms.noise_dynamics = False
for i in range(100000):
  try:
    if i % 10000 == 9999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 100000 == 99999:
        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break


In [ ]:

## turn off noise
__pfc.SetDT(0.001)
__pfc.parms.eta = 0.0
__pfc.parms.noise_dynamics = False
for i in range(100000):
  try:
    if i % 10000 == 9999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 100000 == 99999:
        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
_phi = __pfc.phi.get()
_cm_data = ComputeCM(_phi, __pfc.t, threshold_abs=1.8)
_cm_data

In [ ]:
_mu_epsilon = __pfc.parms.r * _phi

fig = go.Figure(data=go.Heatmap(z=_phi, colorscale='Hot', colorbar_thickness=10))
fig.add_trace(go.Scatter(x=_cm_data[:,2], y=_cm_data[:,1], mode='markers', marker=dict(size=8, symbol='x', opacity=0.2, color='cyan')))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"mu epsilon", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)


In [ ]:
_mu_epsilon = __pfc.parms.r * _phi

layout = dict(hoversubplots='axis', title='mu epsilon', grid=dict(rows=1, columns=2),
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*4 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=1, r=30, t=1, b=1),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',)

fig = go.Figure(layout=layout)
fig.add_trace(go.Heatmap(z=_mu_epsilon, colorscale='hot', xaxis='x', showscale=False))
fig.add_trace(go.Heatmap(z=_phi, colorscale='Hot', xaxis='x2', colorbar=dict(x=-0.15,thickness=10)))
fig.update_layout(
    title=dict(text=f"mu epsilon", font=dict(size=20), x=0.1, y=0.95, yref='paper')
)
fig.show()

In [ ]:
PlotChemicalPotential(__pfc.phi.get(),
                      __pfc.kx.get(),
                      __pfc.ky.get(),
                      __pfc.k2.get(),
                      __pfc.k4.get(),
                      __pfc.t,
                      __pfc.parms.r,
                      __pfc.parms.t,
                      __pfc.parms.b)

In [ ]:
_phi = __pfc.phi.get()
# create a vacancy
_phi[149:158, 61:71] = 0.01

_dphi = PlotChemicalPotential(_phi,
                      __pfc.kx.get(),
                      __pfc.ky.get(),
                      __pfc.k2.get(),
                      __pfc.k4.get(),
                      __pfc.t,
                      __pfc.parms.r,
                      __pfc.parms.t,
                      __pfc.parms.b)

In [ ]:
# create a vacancy
__pfc.phi[149:158, 61:71] = 0.01

# relax (rough)
__pfc.SetDT(0.01)
__pfc.parms.eta = 0.0
__pfc.parms.noise_dynamics = False
for i in range(100000):
  try:
    if i % 10000 == 9999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 100000 == 99999:
        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break
  


In [ ]:
_phi = __pfc.phi.get()

_dphi = PlotChemicalPotential(_phi,
                      __pfc.kx.get(),
                      __pfc.ky.get(),
                      __pfc.k2.get(),
                      __pfc.k4.get(),
                      __pfc.t,
                      __pfc.parms.r,
                      __pfc.parms.t,
                      __pfc.parms.b)

In [ ]:

# relax (fine)
__pfc.SetDT(0.001)
__pfc.parms.eta = 0.0
__pfc.parms.noise_dynamics = False
for i in range(10000):
  try:
    if i % 1000 == 999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//1000:04}.png'))
      if i % 10000 == 9999:
        fig.show()

    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
_phi = __pfc.phi.get()

_dphi = PlotChemicalPotential(_phi,
                      __pfc.kx.get(),
                      __pfc.ky.get(),
                      __pfc.k2.get(),
                      __pfc.k4.get(),
                      __pfc.t,
                      __pfc.parms.r,
                      __pfc.parms.t,
                      __pfc.parms.b)

In [ ]:
# raise the vacancy by decreasing a neighbor (conserve mass)
_phi = __pfc.phi.get()
mass_vacancy = _phi[150:157,62:69].sum()
mass_neighbor = _phi[150:157,72:78].sum()
print(mass_vacancy,mass_neighbor)

# reduce the mass of the neighbor by 70.0% of difference with vacancy
mass_diff = mass_neighbor - mass_vacancy
_phi[150:157,72:78]*=(mass_neighbor - mass_diff*70.0/100)/mass_neighbor

# add the same amount of mass to the vacancy
_phi[150:157,62:69]*=(mass_vacancy + mass_diff*70.0/100)/mass_vacancy

print(__pfc.phi.sum(), _phi.sum())

_dphi = PlotChemicalPotential(_phi,
                      __pfc.kx.get(),
                      __pfc.ky.get(),
                      __pfc.k2.get(),
                      __pfc.k4.get(),
                      __pfc.t,
                      __pfc.parms.r,
                      __pfc.parms.t,
                      __pfc.parms.b)

In [ ]:
# shift neihbor toward the vacancy
_phi = __pfc.phi.get()

_vacancyEdge = _phi[150:157,69:71].copy()
_phi[150:157,69:78]=_phi[150:157,71:80]
_phi[150:157,78:80]=_vacancyEdge

print(__pfc.phi.sum(), _phi.sum())

_dphi = PlotChemicalPotential(_phi,
                      __pfc.kx.get(),
                      __pfc.ky.get(),
                      __pfc.k2.get(),
                      __pfc.k4.get(),
                      __pfc.t,
                      __pfc.parms.r,
                      __pfc.parms.t,
                      __pfc.parms.b)

###### dash

In [ ]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

layout = dict(hoversubplots='axis', title='mu epsilon', grid=dict(rows=1, columns=2))
fig = go.Figure(layout=layout)

# First heatmap
fig.add_trace(go.Heatmap(
    z=_phi,
    x=np.arange(180),
    y=np.arange(180),
    xaxis='x',
    coloraxis="coloraxis1",
    name="Heatmap 1"
))

# Second heatmap
fig.add_trace(go.Heatmap(
    z=_phi,
    x=np.arange(180),
    y=np.arange(180),
    xaxis='x2',
    coloraxis="coloraxis2",
    name="Heatmap 2"
))

fig.update_layout(
    coloraxis1=dict(colorscale="Viridis"),
    coloraxis2=dict(colorscale="Plasma"),
    hovermode="closest"
)

app.layout = html.Div([
    dcc.Graph(id="heatmap", figure=fig),
    html.Div(id="hover-data", style={"margin-top": "20px", "color":"white"}),
])

@app.callback(
    Output("hover-data", "children"),
    Input("heatmap", "hoverData")
)
def display_hover_data(hoverData):
    if hoverData:
        point = hoverData["points"][0]
        return f"Hovered at x={point['x']}, y={point['y']}, z={point['z']}"
    return "Hover over a point!"

if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
import dash
from dash import dcc, html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import json

app = dash.Dash(__name__)

layout = dict(title='mu epsilon', grid=dict(rows=1, columns=2))
fig = go.Figure(layout=layout)

# First heatmap
fig.add_trace(go.Heatmap(
    z=[[i + j for j in range(10)] for i in range(10)],
    x=list(range(10)),
    y=list(range(10)),
    xaxis='x',
    coloraxis="coloraxis1",
    name="Heatmap 1"
))

# Second heatmap
fig.add_trace(go.Heatmap(
    z=[[i * j for j in range(10)] for i in range(10)],
    x=list(range(10)),
    y=list(range(10)),
    xaxis='x2',
    coloraxis="coloraxis2",
    name="Heatmap 2"
))

fig.update_layout(
    coloraxis1=dict(colorscale="Viridis"),
    coloraxis2=dict(colorscale="Plasma"),
    hovermode="closest"
)

app.layout = html.Div([
    dcc.Graph(id="heatmap", figure=fig),
    html.Div(id="hover-data", style={"margin-top": "20px", "color":"white"}),
    html.Script("""
        document.getElementById('heatmap').on('plotly_hover', function(eventdata) {
            var point = eventdata.points[0];
            Plotly.Fx.hover('heatmap', [{curveNumber: 0, pointNumber: point.pointNumber}, {curveNumber: 1, pointNumber: point.pointNumber}]);
        });
    """)
])

@app.callback(
    Output("hover-data", "children"),
    Input("heatmap", "hoverData")
)
def display_hover_data(hoverData):
    if hoverData:
        point = hoverData["points"][0]
        return f"Hovered at x={point['x']}, y={point['y']}, z={point['z']}"
    return "Hover over a point!"

if __name__ == "__main__":
    app.run_server(debug=True)

###### vector fields

In [ ]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

import numpy as np

# x,y = np.meshgrid(np.arange(-2, 2, .2),
# z = x*np.exp(-x**2 - y**2)
v, u = np.gradient(_dphi, .2, .2)

# Create quiver figure
fig = ff.create_quiver(np.meshgrid(np.arange(180)[::5],np.arange(180)[::5])[0],np.meshgrid(np.arange(180)[::5],np.arange(180)[::5])[1], u[::5,::5], v[::5,::5],
                       scale=20000,
                       arrow_scale=.4,
                       name='quiver',
                       line_width=1,
                       line_color='cyan',)
# fig.add_trace(go.Heatmap(z=__pfc.phi.get(), x=__pfc.x.get(), y=__pfc.y.get(), colorscale='hot', colorbar_thickness=10, opacity=0.50))
fig.add_trace(go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10, opacity=0.50))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"dphi", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.show()

In [ ]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

import numpy as np

# x,y = np.meshgrid(np.arange(-2, 2, .2),
# z = x*np.exp(-x**2 - y**2)
v, u = np.gradient(_dphi, .2, .2)

# Create quiver figure
fig = ff.create_streamline(np.arange(180), np.arange(180), _dphi_dy, _dphi_dx,
            arrow_scale=0.4,
            line_width=1,
            line_color='cyan',)
# fig.add_trace(go.Heatmap(z=__pfc.phi.get(), x=__pfc.x.get(), y=__pfc.y.get(), colorscale='hot', colorbar_thickness=10, opacity=0.50))
fig.add_trace(go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10, opacity=0.50))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"dphi", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.show()

### Debug

#### Code review

In [ ]:
_debugPFC = PFC2D_Vacancy()
_debugPFC.parms.r = -3 #-3.1339050579032284
_debugPFC.parms.a = 0
_debugPFC.parms.b = 2 #2.2020489023671126
_debugPFC.parms.t = -1 #-1.1782718056073347
_debugPFC.parms.v = 1
_debugPFC.parms.rho0 = 0.5 #0.5178932522368416

_debugPFC.parms.Hng = 0
_debugPFC.parms.Hln = 1
_debugPFC.parms.N = 20
_debugPFC.parms.PPU = 10
_debugPFC.parms.eta = 0.1
_debugPFC.parms.noise_dynamics = False#True
_debugPFC.parms.dt = 0.01
_debugPFC.parms.seed = 150
_debugPFC.parms.noise_high_freq_cutoff = 1

_debugPFC.InitParms()
# _debugPFC.SetGeometry(80, 72, 10, 1)
_debugPFC.SetGeometry(10, 10, 10, 1)
# _debugPFC.InitFieldCrystal(A = (1.92 - _debugPFC.parms.rho0)/6, noisy = True)
# _debugPFC.phi[32:41,36:45] = 0.00
# _debugPFC.noiseMask[:10,:10] = 0
# _debugPFC.noiseMask[-10:,:10] = 0
# _debugPFC.noiseMask[:10,-10:] = 0
# _debugPFC.noiseMask[-10:,-10:] = 0



In [ ]:
_debugPFC.k2.max()

In [ ]:
_debugPFC.GetEtaNoise()

# plot _rss as heatmap
fig = go.Figure(data=go.Heatmap(z=_debugPFC.noise.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_debugPFC.nx*2 + 100,
    height=_debugPFC.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

_debugPFC.noise, _debugPFC.noise.shape, _debugPFC.noise_fft.shape, _debugPFC.noise_fft

In [ ]:
_debugPFC.InitFieldCrystal(A = (1.92 - _debugPFC.parms.rho0)/6, noisy = false)

# plot _rss as heatmap
fig = go.Figure(data=go.Heatmap(z=_debugPFC.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_debugPFC.nx*2 + 100,
    height=_debugPFC.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)

In [ ]:
print('phi:\n', _debugPFC.phi.T)

In [ ]:
_debugPFC.phi.mean()

In [ ]:
_debugPFC.TimeStepCross()

In [ ]:
print('N0_hat:\n', _debugPFC.N0_hat.T, '\n\nN1_hat:\n', _debugPFC.N1_hat.T, '\n\nphi:\n', _debugPFC.phi.T)

In [ ]:
print('phi2:\n', _debugPFC.N0_hat.T, '\n\nN1_hat:\n', _debugPFC.N1_hat.T, '\n\nphi:\n', _debugPFC.phi.T)

In [ ]:

# plot _rss as heatmap
fig = go.Figure(data=go.Heatmap(z=_debugPFC.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_debugPFC.nx*2 + 100,
    height=_debugPFC.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)

#### Confinement

##### ZF Code

In [ ]:
_zf_Vb0 = 1
_zf_Delta_b = 0.1
_zf_lxly = 256
_zf_r0 = 3*_zf_lxly/8   # circle diameter is 3/4 the width of the grid
_zf_r0

In [ ]:
_zf_x = np.arange(0, _zf_lxly) - _zf_lxly/2 - 0.5
_zf_y = np.arange(0, _zf_lxly) - _zf_lxly/2 - 0.5
_zf_xx, _zf_yy = np.meshgrid(_zf_x, _zf_y)
_zf_rr = np.sqrt(_zf_xx**2 + _zf_yy**2)
_zf_rs = _zf_rr - _zf_r0

In [ ]:
# plot _rss as heatmap
fig = go.Figure(data=go.Heatmap(z=_zf_rs, colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_zf_lxly*2 + 100,
    height=_zf_lxly*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)

In [ ]:
_zf_n_x = _zf_xx/_zf_rr
_zf_n_y = _zf_yy/_zf_rr

In [ ]:
_zf_Vb = _zf_Vb0/(1 + np.exp(-2*_zf_rs/_zf_Delta_b))

In [ ]:
# plot _rss as heatmap
fig = go.Figure(data=go.Heatmap(z=_zf_Vb, colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_zf_lxly*2 + 100,
    height=_zf_lxly*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3.134 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.202 #2.2020489023671126
__pfc.parms.t = -1.178 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.518 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 256
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.noise_dynamics = False#True
__pfc.parms.dt = 0.01
__pfc.parms.seed = 150

__pfc.InitParms()
# __pfc.SetGeometry(80, 72, 10, 1)
__pfc.SetGeometry(256, 256, 10, 1)
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = True)
# __pfc.phi[32:41,36:45] = 0.00
# __pfc.noiseMask[:10,:10] = 0
# __pfc.noiseMask[-10:,:10] = 0
# __pfc.noiseMask[:10,-10:] = 0
# __pfc.noiseMask[-10:,-10:] = 0

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*3 + 100,
    height=__pfc.ny*3 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

# outputPath = f'diffusion_long_medtemp_newnoise{__pfc.parms.seed:03}'
# if not os.path.exists(outputPath):
#   os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(6000):
  try:
    if i % 200 == 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*3 + 100,
          height=__pfc.ny*3 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}", font=dict(size=12), x=0.05, y=0.95, yref='paper')
      )
      if i % 2000 == 1999:
        fig.show()
    #   fig.write_image(os.path.join(outputPath, f'diffusion{i//200:04}.png'))

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

##### My Code

In [ ]:
_Nx = 180
_Ny = 180
_dxdy = 0.726
_x, _y = np.meshgrid(np.arange(_Nx) * __pfc.dx, np.arange(_Ny) * __pfc.dy)
_rs = np.zeros_like(_x)
_center = (_Nx//2 * __pfc.dx, _Ny//2 * __pfc.dy)
_radius = 60 * __pfc.dx
_rss = np.ones_like(_x) * np.where((_x - _center[0])**2 + (_y - _center[1])**2 < _radius**2, -1, 1)


In [ ]:
for i in range(_Nx):
    for j in range(_Ny):
        if _rss[i,j] == 1:
            _rs[i,j] = np.sqrt((_x[_rss == -1] - _x[i, j])**2 + (_y[_rss == -1] - _y[i,j])**2).min()
        else:
            _rs[i,j] = -np.sqrt((_x[_rss == 1] - _x[i, j])**2 + (_y[_rss == 1] - _y[i,j])**2).min()


In [ ]:
_rs

In [ ]:
# plot _rss as heatmap
fig = go.Figure(data=go.Heatmap(z=_rs, colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_Nx*2 + 100,
    height=_Ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)

In [ ]:
_delta = 2*__pfc.dx
_Vb0 = 1.0
_Vb = _Vb0/2 * (1 + np.tanh((_rs)/_delta))
# _Vb[_Vb > 0.1] = 1

In [ ]:
# plot _rss as heatmap
fig = go.Figure(data=go.Surface(z=_Vb, colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_Nx*2 + 100,
    height=_Ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)

In [ ]:
(1-_Vb).sum()/180/180

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 2.2 # 2.2020489023671126
__pfc.parms.t = -1.2 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.51 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 3.0 #1.25
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.001
__pfc.parms.seed = 150
__pfc.parms.Vb = cp.array(_Vb)
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
# __pfc.InitFieldFlat()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi *= 1 - __pfc.parms.Vb / __pfc.parms.Vb.max()
__pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[81:90,85:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

outputPath = f'exp(Confined-soft-boundary-increasing-noise)_rho0!{__pfc.parms.rho0}_eta!{__pfc.parms.eta:0.3f}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(100000):
  try:
    if i % 2000 == 1999:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=100, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}", font=dict(size=12), x=0.05, y=0.95, yref='paper')
      )
      if i % 2000 == 1999:
        fig.show()
        fig.write_image(os.path.join(outputPath, f'diffusion{i//2000:04}.png'))

    # if i % 20000 == 19999:
    #   __pfc.parms.eta*= 1.25
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
for i in range(10000): __pfc.TimeStepCross()
__pfc.CalcEnergyDensity()
print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

#### decreasing stiffness

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 0.1 # 2.2020489023671126
__pfc.parms.t = -1.2 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.60 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 3.0 #1.25
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.001
__pfc.parms.seed = 150
__pfc.parms.Vb = cp.array(_Vb)
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
# __pfc.InitFieldFlat()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi[__pfc.parms.Vb > 0.1] = 0.2
__pfc.parms.Vb *= 60
__pfc.parms.Vb = None
__pfc.phi[116:126,87:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

outputPath = f'exp(Confined-soft-boundary-increasing-noise)_beta_{__pfc.parms.b}_rho0!{__pfc.parms.rho0}_eta!{__pfc.parms.eta:0.3f}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(10000):
  try:
    if i % 200== 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=100, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
      )
      if i % 2000 == 1999:
        fig.show()
        fig.write_image(os.path.join(outputPath, f'diffusion{i//2000:04}.png'))

    # if i % 20000 == 19999:
    #   __pfc.parms.eta*= 1.25
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

#### increasing temp

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -2 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 0.8 # 2.2020489023671126
__pfc.parms.t = -0.9 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.51 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 3.0 #1.25
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.001
__pfc.parms.seed = 150
__pfc.parms.Vb = cp.array(_Vb)
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
# __pfc.InitFieldFlat()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi *= 1 - __pfc.parms.Vb / __pfc.parms.Vb.max()
__pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[81:90,85:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

outputPath = f'exp(Confined-soft-boundary-increasing-noise)_beta_{__pfc.parms.b}_rho0!{__pfc.parms.rho0}_eta!{__pfc.parms.eta:0.3f}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(10000):
  try:
    if i % 200== 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=100, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
      )
      if i % 2000 == 1999:
        fig.show()
        fig.write_image(os.path.join(outputPath, f'diffusion{i//2000:04}.png'))

    # if i % 20000 == 19999:
    #   __pfc.parms.eta*= 1.25
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

#### decreasing g

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 0.4 # 2.2020489023671126
__pfc.parms.t = -0.9 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.51 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 3.0 #1.25
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.001
__pfc.parms.seed = 150
__pfc.parms.Vb = cp.array(_Vb)
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
# __pfc.InitFieldFlat()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi *= 1 - __pfc.parms.Vb / __pfc.parms.Vb.max()
__pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[81:90,85:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

outputPath = f'exp(Confined-soft-boundary-increasing-noise)_beta_{__pfc.parms.b}_rho0!{__pfc.parms.rho0}_eta!{__pfc.parms.eta:0.3f}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(10000):
  try:
    if i % 200== 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=100, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
      )
      if i % 2000 == 1999:
        fig.show()
        fig.write_image(os.path.join(outputPath, f'diffusion{i//2000:04}.png'))

    # if i % 20000 == 19999:
    #   __pfc.parms.eta*= 1.25
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -3 #-3.1339050579032284
__pfc.parms.a = 0
__pfc.parms.b = 0.1 # 2.2020489023671126
__pfc.parms.t = -1.2 #-1.1782718056073347
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.51 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 3.0 #1.25
__pfc.parms.noise_dynamics = True
__pfc.parms.dt = 0.001
__pfc.parms.seed = 150
__pfc.parms.Vb = cp.array(_Vb)
__pfc.parms.phi_b = 0.00

__pfc.InitParms()
__pfc.SetGeometry(180, 180, 10, 1)
# __pfc.InitFieldFlat()
__pfc.InitFieldCrystal(A = (1.92 - __pfc.parms.rho0)/6, noisy = False)
__pfc.phi *= 1 - __pfc.parms.Vb / __pfc.parms.Vb.max()
__pfc.parms.Vb *= 60
# __pfc.parms.Vb = None
__pfc.phi[81:90,85:95] = 0.00

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=100, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
)
fig.show()
__pfc.CalcEnergyDensity()

print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')

outputPath = f'exp(Confined-soft-boundary-increasing-noise)_beta_{__pfc.parms.b}_rho0!{__pfc.parms.rho0}_eta!{__pfc.parms.eta:0.3f}_seed{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(10000):
  try:
    if i % 200== 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=100, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}, eta: {__pfc.parms.eta:.3f}", font=dict(size=12))
      )
      if i % 2000 == 1999:
        fig.show()
        fig.write_image(os.path.join(outputPath, f'diffusion{i//2000:04}.png'))

    # if i % 20000 == 19999:
    #   __pfc.parms.eta*= 1.25
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

#### t

In [ ]:
__pfc = PFC2D_Vacancy()
__pfc.parms.r = -2.6
__pfc.parms.a = 0
__pfc.parms.b = 1
__pfc.parms.t = -2
__pfc.parms.v = 1
__pfc.parms.rho0 = 0.1 #0.5178932522368416

__pfc.parms.Hng = 0
__pfc.parms.Hln = 1
__pfc.parms.N = 180
__pfc.parms.PPU = 10
__pfc.parms.eta = 0.1
__pfc.parms.noise_dynamics = False
__pfc.parms.dt = 0.011262238533095706
__pfc.parms.seed = 150

__pfc.InitParms()
# __pfc.SetGeometry(80, 72, 10, 1)
__pfc.InitFieldFlat()
# __pfc.noiseMask[:10,:10] = 0
# __pfc.noiseMask[-10:,:10] = 0
# __pfc.noiseMask[:10,-10:] = 0
# __pfc.noiseMask[-10:,-10:] = 0

fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=__pfc.nx*2 + 100,
    height=__pfc.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

print(__pfc.t, __pfc.phi.mean(), __pfc.phi.min(), __pfc.phi.max(), np.power((__pfc.phi - __pfc.phi0),2).max())

outputPath = f'diffusion_long_medtemp_newnoise{__pfc.parms.seed:03}'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

# show the image scaled to 1 px per point
for i in range(6000):
  try:
    if i % 200 == 199:
      __pfc.CalcEnergyDensity()
      print(f'f: {__pfc.f:.10f}, t: {__pfc.t:.2f}, min: {__pfc.phi.min()}, max: {__pfc.phi.max()}, mean: {__pfc.phi.mean()}')
       
      fig = go.Figure(data=go.Heatmap(z=__pfc.phi.get(), colorscale='hot', colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=__pfc.nx*2 + 100,
          height=__pfc.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {__pfc.t:.6f}", font=dict(size=12), x=0.05, y=0.95, yref='paper')
      )
      if i % 2000 == 1999:
        fig.show()
      fig.write_image(os.path.join(outputPath, f'diffusion{i//200:04}.png'))

    # if i % 5 == 2:
    #   __pfc.AddNoise()
    
    __pfc.TimeStepCross()
  except Exception as e:
    print(e)
    break

#### Stuff

In [ ]:
pfc = PFC2D_Vacancy()
pfc.parms.r = -1.6
pfc.parms.N = 100
pfc.parms.PPU = 10
pfc.parms.rho0 = 0.3
pfc.parms.eta = 0.7
pfc.parms.dt = 1e-2

pfc.InitParms()
pfc.InitFieldCrystal()
pfc.phi_hat[0,0]

In [ ]:
pfc.lincoeff[0,0], pfc.expcoeff[0,0], pfc.expcoeff_nonlin[0,0], pfc.expcoeff_nonlin2[0,0]

In [ ]:
pfc.t, pfc.phi.mean(), pfc.phi.min(), pfc.phi.max(), np.power((pfc.phi - pfc.phi0),2).max()

In [ ]:
pfc.phi = _phi
pfc.phi_hat = _phi_hat
pfc.t = _t
pfc.parms.dt = 3e-10 # worked at 2.6 broke at 2.7
for i in range(300):
  pfc.TimeStep()
  pfc.phi_hat[50,50]=0
PlotTimeStep(pfc)
print(pfc.phi.min(), pfc.phi.max(), pfc.phi.mean(), pfc.t)

In [ ]:
pfc.parms.dt = 3e-10 # worked at 2.6 broke at 2.7
for i in range(1):
  pfc.TimeStep()
PlotTimeStep(pfc)
print(pfc.phi.min(), pfc.phi.max(), pfc.phi.mean(), pfc.t)

In [ ]:
plt.imshow(np.log(np.abs(pfc.phi_hat)))

In [ ]:
pfc.phi_hat[50,50]

In [ ]:
pfc.kx[50,49:52], pfc.ky[50,49:52]

In [ ]:
pfc.TimeStep()
pfc.lincoeff[49:52,49:52], -pfc.k2[49:52,49:52], pfc.phi_hat[49:52,49:52], pfc.phi3_hat[49:52,49:52]

In [ ]:
pfc.lincoeff

In [ ]:
expcoeff = np.exp(pfc.lincoeff*pfc.parms.dt)
expcoeff.min(), expcoeff.max()

expcoeff_nonlin = np.ones_like(expcoeff) * pfc.parms.dt
expcoeff_nonlin[pfc.lincoeff != 0] = (expcoeff[pfc.lincoeff != 0] - 1)/pfc.lincoeff[pfc.lincoeff != 0]
expcoeff_nonlin.min(), expcoeff_nonlin.max()

In [ ]:
expcoeff_nonlin = np.ones_like(expcoeff) * pfc.parms.dt
expcoeff_nonlin[pfc.lincoeff != 0] = (expcoeff[pfc.lincoeff != 0] - 1)/pfc.lincoeff[pfc.lincoeff != 0]
expcoeff_nonlin.min(), expcoeff_nonlin.max()

In [ ]:
(np.exp(0.00000001*0.1) - (1+0.00000001*0.1))/0.00000001**2

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Phi")
plt.contourf(pfc.phi, pfc.nx, cmap='viridis')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("Normalized Gaussian White Noise")
plt.contourf(pfc.GetEtaNoise(), pfc.nx, cmap='viridis')
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
plt.contourf(pfc.k4, pfc.nx, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
plt.contourf(pfc.lincoeff, pfc.nx, cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
pfc.TimeStep()

In [ ]:
pfc.phi_hat.shape

In [ ]:
pfc.phi_hat.shape

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Phi")
plt.contourf(pfc.phi, pfc.nx, cmap='viridis')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title("Normalized Gaussian White Noise")
plt.contourf(pfc.GetEtaNoise(), pfc.nx, cmap='viridis')
plt.colorbar()

plt.tight_layout()
plt.show()

In [ ]:
_x, _y, m, n = symbols('x_0 y m n')
f, g = symbols('f g')
x = _x
y = _y
f = x*y
g = x**y
exp_list = [f, g]



In [ ]:
exp_list

In [ ]:
for i, exp in enumerate(exp_list):
    exp = exp.subs(_x, _x + 1)
    exp = exp.subs(_y, _y + 1)
    exp_list[i] = exp

exp_list